In [1]:
from sklearn.cluster import KMeans
import numpy as np
import matplotlib.pyplot as plt

from matplotlib import cm
import matplotlib.cbook as cbook
import matplotlib.colors as colors

import pandas as pd
import xarray as xr
from netCDF4 import Dataset, num2date, date2num
from datetime import datetime, timedelta
import os

import seaborn as sns
import cmocean as cmocn

import dask
from dask import delayed

%matplotlib inline

In [2]:
ds= xr.open_dataset('/srv/data2/srai_poseidon/srai_poseidon/observation/SatelliteVsBuoy/downloads/Buoy/extractedGZ/WINDS/T_09N_140W_xrr_MatchUp_2000.nc')
ds

<xarray.Dataset>
Dimensions:                           (QS_TIME: 6044, cross_track: 3,
                                       ambiguities: 4, TIME: 6762,
                                       TAO_TIME_INDEX: 73, HEIGHT: 3, DEPTH: 1)
Coordinates:
  * QS_TIME                           (QS_TIME) datetime64[ns] 2000-01-01T03:...
  * cross_track                       (cross_track) int64 0 1 2
  * ambiguities                       (ambiguities) int64 0 1 2 3
  * TIME                              (TIME) datetime64[ns] 2000-01-01T03:51:...
  * TAO_TIME_INDEX                    (TAO_TIME_INDEX) int64 -36 -35 ... 35 36
  * HEIGHT                            (HEIGHT) float32 3.0 4.0 10.0
  * DEPTH                             (DEPTH) float32 1.0
Data variables: (12/41)
    retrieved_wind_speed              (QS_TIME, cross_track) float64 ...
    retrieved_wind_direction          (QS_TIME, cross_track) float64 ...
    rain_impact                       (QS_TIME, cross_track) float64 ...
    flags                             (QS_TIME, cross_track) float64 ...
    eflags                            (QS_TIME, cross_track) float64 ...
    nudge_wind_speed                  (QS_TIME, cross_track) float64 ...
    ...                                ...
    RELH                              (QS_TIME, TAO_TIME_INDEX, HEIGHT) float32 ...
    RELH_QC                           (QS_TIME, TAO_TIME_INDEX, HEIGHT) float32 ...
    RELH_DM                           (QS_TIME, TAO_TIME_INDEX, HEIGHT) |S1 ...
    WSPD_10N                          (QS_TIME, TAO_TIME_INDEX, HEIGHT) float64 ...
    WSPD_10                           (QS_TIME, TAO_TIME_INDEX, HEIGHT) float64 ...
    TAO_TIME                          (QS_TIME, TAO_TIME_INDEX) datetime64[ns] ...

In [5]:
100/24

4.166666666666667

In [4]:
list(ds.keys())

['retrieved_wind_speed',
 'retrieved_wind_direction',
 'rain_impact',
 'flags',
 'eflags',
 'nudge_wind_speed',
 'nudge_wind_direction',
 'retrieved_wind_speed_uncorrected',
 'cross_track_wind_speed_bias',
 'atmospheric_speed_bias',
 'wind_obj',
 'ambiguity_speed',
 'ambiguity_direction',
 'ambiguity_obj',
 'number_in_fore',
 'number_in_aft',
 'number_out_fore',
 'number_out_aft',
 'gmf_sst',
 'distance_from_coast',
 'exp_bias_wrt_oceanward_neighbors',
 'WDIR',
 'WDIR_QC',
 'WDIR_DM',
 'WSPD',
 'WSPD_QC',
 'WSPD_DM',
 'UWND',
 'VWND',
 'SST',
 'SST_QC',
 'SST_DM',
 'AIRT',
 'AIRT_QC',
 'AIRT_DM',
 'RELH',
 'RELH_QC',
 'RELH_DM',
 'WSPD_10N',
 'WSPD_10',
 'TAO_TIME']

In [5]:
def getSubDF(ds, LAT, LON):
    subds = xr.Dataset()

    taoTimeVar = ['TAO_TIME']
    
    windVars = ['WDIR',
         'WDIR_QC',
         'WDIR_DM',
         'WSPD',
         'WSPD_QC',
         'WSPD_DM',
         'UWND',
         'VWND']

    sstVars = ['SST',
             'SST_QC',
             'SST_DM']
    
    airtVars = ['AIRT',
             'AIRT_QC',
             'AIRT_DM']
    
    rhVars = ['RELH',
             'RELH_QC',
             'RELH_DM']
    
    u10Vars =['WSPD_10N',
             'WSPD_10']
    
    
    satVars = ['retrieved_wind_speed',
             'retrieved_wind_direction',
             'rain_impact',
             'flags',
             'eflags',
             'nudge_wind_speed',
             'nudge_wind_direction',
             'retrieved_wind_speed_uncorrected',
             'cross_track_wind_speed_bias',
             'atmospheric_speed_bias',
             'wind_obj',
             'ambiguity_speed',
             'ambiguity_direction',
             'ambiguity_obj',
             'number_in_fore',
             'number_in_aft',
             'number_out_fore',
             'number_out_aft',
             'gmf_sst',
             'distance_from_coast',
             'exp_bias_wrt_oceanward_neighbors']

     
    
    for windVar in windVars:
        subds[windVar] = ds[windVar].sel(HEIGHT=4)
    subds.drop_vars('HEIGHT')
    for sstVar in sstVars:
        subds[sstVar] = ds[sstVar].sel(DEPTH=1)
    subds.drop_vars('DEPTH')
    for rhVar in rhVars:
        subds[rhVar] = ds[rhVar].sel(HEIGHT=3)
    subds.drop_vars('HEIGHT')
    for airtVar in airtVars:
        subds[airtVar] = ds[airtVar].sel(HEIGHT=3)
    subds.drop_vars('HEIGHT')
    for u10Var in u10Vars:
        subds[u10Var] = ds[u10Var].sel(HEIGHT=10)
    subds.drop_vars('HEIGHT')

    subds['LATITUDE'] = xr.DataArray(LAT * np.ones(len(subds['QS_TIME']), dtype = float), dims=['QS_TIME'])
    subds['LONGITUDE'] = xr.DataArray(LON * np.ones(len(subds['QS_TIME']), dtype = float), dims=['QS_TIME'])
    
    subds['TAO_TIME'] = ds['TAO_TIME']

    deployFileName = f'../../downloads/Buoy/extractedGZ/WINDS/T_{lat:02d}{latUnits}_{lon:03d}{lonUnits}/T_{lat:02d}{latUnits}_{lon:03d}{lonUnits}_DeploymentDates.nc'
    ds2 = xr.open_dataset(deployFileName)
    startDates = np.array(ds2['startDate'])
    endDates = np.array(ds2['endDate'])

    depNum = np.zeros((len(subds['QS_TIME'])), dtype=int)
    c = 1
    for i in range(1,len(startDates)):
        mask = subds['QS_TIME'] >= startDates[i]
        mask *= subds['QS_TIME'] <= endDates[i]
        depNum[mask] = c
        c = c+1

    subds['Deployment index'] = xr.DataArray(depNum, dims=['QS_TIME'])
    
    
    satDS = ds[satVars]
    satDS = satDS.isel(cross_track=0, ambiguities=0)

    
    
    returnDS = xr.merge((subds, satDS))

    
    
    return returnDS

In [6]:
latList = [-9, -8, -5, -2, 0, 2, 5, 8, 9]
lonList = [-95, -110, -125, -140, -155, -170, -180, 165]

ylen = len(latList)
xlen = len(lonList)

taskList = []

for latId  in range(ylen):
    for lonId in range(xlen):
        taskList.append([latList[latId], lonList[lonId]])

ntasks = len(taskList)


allDS = xr.Dataset()
dataCount = 0
fileCount = 0
for task in taskList:
    lat = task[0]
    lon = task[1]

    LAT = lat
    LON = lon

    if lat < 0:
        latUnits = 'S'
    else:
        latUnits = 'N'

    if lon < 0:
        lonUnits = 'W'
    else:
        lonUnits = 'E'

    LON = (LON+360)%360
    lat=abs(lat)
    lon=abs(lon)

    matchFname = f'../../downloads/Buoy/extractedGZ/WINDS/T_{lat:02d}{latUnits}_{lon:03d}{lonUnits}_xrr_MatchUp_2000.nc'
    #deployFileName = f'../../downloads/Buoy/extractedGZ/WINDS/T_{lat:02d}{latUnits}_{lon:03d}{lonUnits}/T_{lat:02d}{latUnits}_{lon:03d}{lonUnits}_DeploymentDates.nc'
    
    if os.path.isfile(matchFname):
        print(f'T_{lat:02d}{latUnits}_{lon:03d}{lonUnits}_xrr_MatchUp_2000.nc')
        rds = xr.open_dataset(matchFname)
        ds = getSubDF(rds, LAT, LON)
        QS_TIME = ds['QS_TIME'].to_numpy()
        ds = ds.rename({'QS_TIME':'count'})
        dataLen = len(ds['count'])
        print(dataCount, dataCount+dataLen)
        ds['count'] = xr.DataArray(pd.Series(np.arange(dataCount, dataCount+dataLen)), 
                                   dims = ['count'])
        ds['QS_TIME'] = xr.DataArray(QS_TIME, 
                                   dims = ['count'])
        dataCount += dataLen
        if fileCount == 0:
            allDS = ds
        else:
            allDS = xr.concat((allDS, ds), dim='count')
        fileCount += 1
        

T_08S_095W_xrr_MatchUp_2000.nc
0 4091
T_08S_110W_xrr_MatchUp_2000.nc
4091 7839


/home/shikhar.rai/anaconda3/envs/mpiNetCDF/lib/python3.11/site-packages/xarray/core/concat.py:546: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  common_dims = tuple(pd.unique([d for v in vars for d in v.dims]))
/home/shikhar.rai/anaconda3/envs/mpiNetCDF/lib/python3.11/site-packages/xarray/core/concat.py:546: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  common_dims = tuple(pd.unique([d for v in vars for d in v.dims]))


T_08S_125W_xrr_MatchUp_2000.nc
7839 12311


/home/shikhar.rai/anaconda3/envs/mpiNetCDF/lib/python3.11/site-packages/xarray/core/concat.py:546: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  common_dims = tuple(pd.unique([d for v in vars for d in v.dims]))
/home/shikhar.rai/anaconda3/envs/mpiNetCDF/lib/python3.11/site-packages/xarray/core/concat.py:546: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  common_dims = tuple(pd.unique([d for v in vars for d in v.dims]))


T_08S_155W_xrr_MatchUp_2000.nc
12311 15908


...

/home/shikhar.rai/anaconda3/envs/mpiNetCDF/lib/python3.11/site-packages/xarray/core/concat.py:546: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  common_dims = tuple(pd.unique([d for v in vars for d in v.dims]))
/home/shikhar.rai/anaconda3/envs/mpiNetCDF/lib/python3.11/site-packages/xarray/core/concat.py:546: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  common_dims = tuple(pd.unique([d for v in vars for d in v.dims]))


T_08N_165E_xrr_MatchUp_2000.nc
233017 238343


/home/shikhar.rai/anaconda3/envs/mpiNetCDF/lib/python3.11/site-packages/xarray/core/concat.py:546: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  common_dims = tuple(pd.unique([d for v in vars for d in v.dims]))
/home/shikhar.rai/anaconda3/envs/mpiNetCDF/lib/python3.11/site-packages/xarray/core/concat.py:546: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  common_dims = tuple(pd.unique([d for v in vars for d in v.dims]))


T_09N_140W_xrr_MatchUp_2000.nc
238343 244387


/home/shikhar.rai/anaconda3/envs/mpiNetCDF/lib/python3.11/site-packages/xarray/core/concat.py:546: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  common_dims = tuple(pd.unique([d for v in vars for d in v.dims]))
/home/shikhar.rai/anaconda3/envs/mpiNetCDF/lib/python3.11/site-packages/xarray/core/concat.py:546: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  common_dims = tuple(pd.unique([d for v in vars for d in v.dims]))


In [7]:
allDS = allDS.drop_vars(['HEIGHT','DEPTH','cross_track','ambiguities'])

In [8]:
allDS

<xarray.Dataset>
Dimensions:                           (count: 244387, TAO_TIME_INDEX: 73)
Coordinates:
  * count                             (count) int64 0 1 2 ... 244385 244386
  * TAO_TIME_INDEX                    (TAO_TIME_INDEX) int64 -36 -35 ... 35 36
Data variables: (12/45)
    WDIR                              (count, TAO_TIME_INDEX) float32 nan ......
    WDIR_QC                           (count, TAO_TIME_INDEX) float32 nan ......
    WDIR_DM                           (count, TAO_TIME_INDEX) |S1 b'' ... b'D'
    WSPD                              (count, TAO_TIME_INDEX) float32 nan ......
    WSPD_QC                           (count, TAO_TIME_INDEX) float32 nan ......
    WSPD_DM                           (count, TAO_TIME_INDEX) |S1 b'' ... b'D'
    ...                                ...
    number_out_fore                   (count) float64 6.0 7.0 7.0 ... 16.0 14.0
    number_out_aft                    (count) float64 8.0 7.0 9.0 ... 26.0 26.0
    gmf_sst                           (count) float64 23.89 23.79 ... 26.45
    distance_from_coast               (count) float64 864.0 876.0 ... 1.886e+03
    exp_bias_wrt_oceanward_neighbors  (count) float64 0.0 0.0 0.0 ... 0.0 0.0
    QS_TIME                           (count) datetime64[ns] 2000-01-01T00:34...

In [9]:
extended_flag_meaning = ['rain_correction_not_applied_flag',
'correction_produced_negative_spd_flag',
'all_ambiguities_contribute_to_nudging_flag',
'large_rain_correction_flag',
'coastal_processing_applied_flag',
'lake_winds_flag',
'rain_nearby_flag',
'ice_nearby_flag',
'significant_rain_correction_flag',
'rain_correction_applied_flag',
'wind_retrieval_possibly_corrupted_flag']

extended_flags = np.array(allDS['eflags'].to_numpy(), dtype=int)
eflags = np.array(['{:016b}'.format(val) for val in extended_flags])
rainCorrectionNotApplied = np.array([val[15] == '0' for val in eflags])
rainCorrectionApplied = np.array([val[4] == '1' for val in eflags])
significantRainCorrectionApplied = np.array([val[5] == '1' for val in eflags])
largeRainCorrectionApplied = np.array([val[12] == '1' for val in eflags])
rainMask = np.logical_or(rainCorrectionApplied, significantRainCorrectionApplied)
rainMask = np.logical_or(rainMask, largeRainCorrectionApplied)
rainMask = np.logical_or(rainMask, rainCorrectionNotApplied)

In [10]:
noRainMask = ~rainMask
mask1 = allDS.sel(TAO_TIME_INDEX=0)['WSPD_QC'].isin([1,2]).to_numpy()
mask2 = allDS.sel(TAO_TIME_INDEX=0)['WDIR_QC'].isin([1,2]).to_numpy()
mask3 = allDS.sel(TAO_TIME_INDEX=0)['SST_QC'].isin([1,2]).to_numpy()
mask4 = allDS.sel(TAO_TIME_INDEX=0)['RELH_QC'].isin([1,2]).to_numpy()
mask5 = allDS.sel(TAO_TIME_INDEX=0)['AIRT_QC'].isin([1,2]).to_numpy()

In [11]:
selectMask = np.logical_and(noRainMask, mask1)
selectMask = np.logical_and(selectMask, mask2)
selectMask = np.logical_and(selectMask, mask3)
selectMask = np.logical_and(selectMask, mask4)
selectMask = np.logical_and(selectMask, mask5)

In [12]:
selectMask

array([ True,  True,  True, ...,  True,  True,  True])

In [13]:
indices = selectMask.nonzero()[0]

In [14]:
goodDataSet = allDS.sel(count=indices)

In [15]:
goodDataSet

<xarray.Dataset>
Dimensions:                           (count: 156953, TAO_TIME_INDEX: 73)
Coordinates:
  * count                             (count) int64 0 1 2 ... 244385 244386
  * TAO_TIME_INDEX                    (TAO_TIME_INDEX) int64 -36 -35 ... 35 36
Data variables: (12/45)
    WDIR                              (count, TAO_TIME_INDEX) float32 nan ......
    WDIR_QC                           (count, TAO_TIME_INDEX) float32 nan ......
    WDIR_DM                           (count, TAO_TIME_INDEX) |S1 b'' ... b'D'
    WSPD                              (count, TAO_TIME_INDEX) float32 nan ......
    WSPD_QC                           (count, TAO_TIME_INDEX) float32 nan ......
    WSPD_DM                           (count, TAO_TIME_INDEX) |S1 b'' ... b'D'
    ...                                ...
    number_out_fore                   (count) float64 6.0 7.0 7.0 ... 16.0 14.0
    number_out_aft                    (count) float64 8.0 7.0 9.0 ... 26.0 26.0
    gmf_sst                           (count) float64 23.89 23.79 ... 26.45
    distance_from_coast               (count) float64 864.0 876.0 ... 1.886e+03
    exp_bias_wrt_oceanward_neighbors  (count) float64 0.0 0.0 0.0 ... 0.0 0.0
    QS_TIME                           (count) datetime64[ns] 2000-01-01T00:34...

In [19]:
rechunk = {'count' : 100}
goodDataSet = goodDataSet.chunk(rechunk)

In [26]:
len(goodDataSet['count'])

156953

In [37]:
def calMeanAndStd_np(arr):
    return np.nanmean(arr), np.nanmean(std)

In [77]:
def getMeanAndStdDev(ds, timeWindow):
    TAO_TIME = np.array(ds['TAO_TIME'].to_numpy(), dtype='datetime64[ns]')[0,:]
    QS_TIME = np.array(ds['QS_TIME'].to_numpy(), dtype='datetime64[ns]')
    mask = np.isnat(TAO_TIME)
    #ds = ds.isel(QS_TIME=0)
    ds = ds.isel(TAO_TIME_INDEX = ~mask)
    TAO_TIME = np.array(ds['TAO_TIME'].to_numpy(), dtype='datetime64[ns]')
    timeDiffInMins = np.array(TAO_TIME - QS_TIME, dtype='timedelta64[s]')/60
    absTimeDiffInMins = np.array(abs(timeDiffInMins), dtype=np.float32)
    mask = absTimeDiffInMins <= timeWindow/2
    # mask = np.logical_and(mask, ds.sel(HEIGHT=4)['WSPD_QC'].isin([1,2]).to_numpy())
    # mask = np.logical_and(mask, ds.sel(HEIGHT=4)['WDIR_QC'].isin([1,2]).to_numpy())
    # mask = np.logical_and(mask, ds.sel(DEPTH=1)['SST_QC'].isin([1,2]).to_numpy())
    # mask = np.logical_and(mask, ds.sel(HEIGHT=3)['AIRT_QC'].isin([1,2]).to_numpy())
    # mask = np.logical_and(mask, ds.sel(HEIGHT=3)['RELH_QC'].isin([1,2]).to_numpy())
    
    # sel_WSPD = ds.sel(HEIGHT = 4.0)['WSPD'].to_numpy()[mask]
    # sel_WSPD_10N = ds.sel(HEIGHT = 10.0)['WSPD_10N'].to_numpy()[mask]
    # sel_WDIR = ds.sel(HEIGHT = 4.0)['WDIR'].to_numpy()[mask]

    mask = np.logical_and(mask, ds['WSPD_QC'].isin([1,2]).to_numpy())
    mask = np.logical_and(mask, ds['WDIR_QC'].isin([1,2]).to_numpy())
    mask = np.logical_and(mask, ds['SST_QC'].isin([1,2]).to_numpy())
    mask = np.logical_and(mask, ds['AIRT_QC'].isin([1,2]).to_numpy())
    mask = np.logical_and(mask, ds['RELH_QC'].isin([1,2]).to_numpy())
    
    sel_WSPD = ds['WSPD'].to_numpy()[mask]
    sel_WSPD_10N = ds['WSPD_10N'].to_numpy()[mask]
    sel_WDIR = ds['WDIR'].to_numpy()[mask]

    sel_Ux_10N = sel_WSPD_10N * np.cos(np.deg2rad((-(sel_WDIR - 90.0) + 360)%360))
    sel_Vy_10N = sel_WSPD_10N * np.sin(np.deg2rad((-(sel_WDIR - 90.0) + 360)%360))

    sel_cosWDIR = np.cos(np.deg2rad((-(sel_WDIR - 90.0) + 360)%360))
    sel_sinWDIR = np.sin(np.deg2rad((-(sel_WDIR - 90.0) + 360)%360))

    if np.sum(mask) > 0:
        stdWSPD = np.nanstd(sel_WSPD)
        meanWSPD = np.nanmean(sel_WSPD)

        stdWSPD_10N = np.nanstd(sel_WSPD_10N)
        meanWSPD_10N = np.nanmean(sel_WSPD_10N)

        stdUx_10N = np.nanstd(sel_Ux_10N)
        meanUx_10N = np.nanmean(sel_Ux_10N)

        stdVy_10N = np.nanstd(sel_Vy_10N)
        meanVy_10N = np.nanmean(sel_Vy_10N)
    
        stdWDIR = np.nanstd(sel_WDIR)
        meanWDIR = np.nanmean(sel_WDIR)

        stdCosWDIR = np.nanstd(sel_cosWDIR)
        meanCosWDIR = np.nanmean(sel_cosWDIR)

        stdSinWDIR = np.nanstd(sel_sinWDIR)
        meanSinWDIR = np.nanmean(sel_sinWDIR)
    else:
        stdWSPD = np.nan
        meanWSPD = np.nan

        stdWSPD_10N = np.nan
        meanWSPD_10N = np.nan

        stdUx_10N = np.nan
        meanUx_10N = np.nan

        stdVy_10N = np.nan
        meanVy_10N = np.nan
    
        stdWDIR = np.nan
        meanWDIR = np.nan

        stdCosWDIR = np.nan
        meanCosWDIR = np.nan

        stdSinWDIR = np.nan
        meanSinWDIR = np.nan
    
    return meanWSPD, stdWSPD, meanWSPD_10N, stdWSPD_10N, stdUx_10N, meanUx_10N, stdVy_10N, meanVy_10N, meanWDIR, stdWDIR, meanCosWDIR, stdCosWDIR, meanSinWDIR, stdSinWDIR

def makeMeanAndStdXarrVars(dummyDS, timeWindowInMins):
    meanWSPD, stdWSPD, meanWSPD_10N, stdWSPD_10N, stdUx_10N, meanUx_10N, stdVy_10N, meanVy_10N, meanWDIR, stdWDIR, meanCosWDIR, stdCosWDIR, meanSinWDIR, stdSinWDIR = getMeanAndStdDev(dummyDS, timeWindowInMins)

    dummyDS[f'mean WSPD {timeWindowInMins:d}min'] = xr.DataArray([meanWSPD], dims = ['count'], 
                                                attrs = {'units' : 'm/s', 'long_name': f'{timeWindowInMins} min centered mean'})
    dummyDS[f'std. dev. WSPD {timeWindowInMins:d}min'] = xr.DataArray([stdWSPD], dims = ['count'], 
                                                attrs = {'units' : 'm/s', 'long_name': f'{timeWindowInMins} min centered std. dev.'})
    
    dummyDS[f'mean WSPD_10N {timeWindowInMins:d}min'] = xr.DataArray([meanWSPD_10N], dims = ['count'], 
                                                attrs = {'units' : 'm/s', 'long_name': f'{timeWindowInMins} min centered mean'})
    dummyDS[f'std. dev. WSPD_10N {timeWindowInMins:d}min'] = xr.DataArray([stdWSPD_10N], dims = ['count'], 
                                                attrs = {'units' : 'm/s', 'long_name': f'{timeWindowInMins} min centered std. dev.'})
    
    dummyDS[f'mean Ux_10N {timeWindowInMins:d}min'] = xr.DataArray([meanUx_10N], dims = ['count'], 
                                                attrs = {'units' : 'm/s', 'long_name': f'{timeWindowInMins} min centered mean'})
    dummyDS[f'std. dev. Ux_10N {timeWindowInMins:d}min'] = xr.DataArray([stdUx_10N], dims = ['count'], 
                                                attrs = {'units' : 'm/s', 'long_name': f'{timeWindowInMins} min centered std. dev.'})
    
    dummyDS[f'mean Vy_10N {timeWindowInMins:d}min'] = xr.DataArray([meanVy_10N], dims = ['count'], 
                                                attrs = {'units' : 'm/s', 'long_name': f'{timeWindowInMins} min centered mean'})
    dummyDS[f'std. dev. Vy_10N {timeWindowInMins:d}min'] = xr.DataArray([stdVy_10N], dims = ['count'], 
                                                attrs = {'units' : 'm/s', 'long_name': f'{timeWindowInMins} min centered std. dev.'})
    
    dummyDS[f'mean WDIR {timeWindowInMins:d}min'] = xr.DataArray([meanWDIR], dims = ['count'], 
                                                attrs = {'units' : 'degrees (True)', 'long_name': f'{timeWindowInMins} min centered mean'})
    dummyDS[f'std. dev. WDIR {timeWindowInMins:d}min'] = xr.DataArray([stdWDIR], dims = ['count'], 
                                                attrs = {'units' : 'degrees (True)', 'long_name': f'{timeWindowInMins} min centered std. dev'})
    
    dummyDS[f'mean cosWDIR {timeWindowInMins:d}min'] = xr.DataArray([meanCosWDIR], dims = ['count'], 
                                                attrs = {'units' : '', 'long_name': f'{timeWindowInMins} min centered mean'})
    dummyDS[f'std. dev. cosWDIR {timeWindowInMins:d}min'] = xr.DataArray([stdCosWDIR], dims = ['count'], 
                                                attrs = {'units' : '', 'long_name': f'{timeWindowInMins} min centered std. dev'})
    
    dummyDS[f'mean sinWDIR {timeWindowInMins:d}min'] = xr.DataArray([meanSinWDIR], dims = ['count'], 
                                                attrs = {'units' : '', 'long_name': f'{timeWindowInMins} min centered mean'})
    dummyDS[f'std. dev. sinWDIR {timeWindowInMins:d}min'] = xr.DataArray([stdSinWDIR], dims = ['count'], 
                                                attrs = {'units' : '', 'long_name': f'{timeWindowInMins} min centered std. dev'})
    return dummyDS


def makeEmptyMeanAndStdXarrVars(dummyDS, timeWindowInMins):
    meanWSPD = np.zeros( (len(dummyDS['count'])), dtype=np.float32) 
    stdWSPD = np.zeros( (len(dummyDS['count'])), dtype=np.float32) 
    meanWSPD_10N = np.zeros( (len(dummyDS['count'])), dtype=np.float32) 
    stdWSPD_10N = np.zeros( (len(dummyDS['count'])), dtype=np.float32) 
    stdUx_10N = np.zeros( (len(dummyDS['count'])), dtype=np.float32) 
    meanUx_10N = np.zeros( (len(dummyDS['count'])), dtype=np.float32) 
    stdVy_10N = np.zeros( (len(dummyDS['count'])), dtype=np.float32) 
    meanVy_10N = np.zeros( (len(dummyDS['count'])), dtype=np.float32) 
    meanWDIR = np.zeros( (len(dummyDS['count'])), dtype=np.float32) 
    stdWDIR = np.zeros( (len(dummyDS['count'])), dtype=np.float32) 
    meanCosWDIR = np.zeros( (len(dummyDS['count'])), dtype=np.float32) 
    stdCosWDIR = np.zeros( (len(dummyDS['count'])), dtype=np.float32) 
    meanSinWDIR = np.zeros( (len(dummyDS['count'])), dtype=np.float32) 
    stdSinWDIR = np.zeros( (len(dummyDS['count'])), dtype=np.float32)

    dummyDS[f'mean WSPD {timeWindowInMins:d}min'] = xr.DataArray(meanWSPD, dims = ['count'], 
                                                attrs = {'units' : 'm/s', 'long_name': f'{timeWindowInMins} min centered mean'})
    dummyDS[f'std. dev. WSPD {timeWindowInMins:d}min'] = xr.DataArray(stdWSPD, dims = ['count'], 
                                                attrs = {'units' : 'm/s', 'long_name': f'{timeWindowInMins} min centered std. dev.'})
    
    dummyDS[f'mean WSPD_10N {timeWindowInMins:d}min'] = xr.DataArray(meanWSPD_10N, dims = ['count'], 
                                                attrs = {'units' : 'm/s', 'long_name': f'{timeWindowInMins} min centered mean'})
    dummyDS[f'std. dev. WSPD_10N {timeWindowInMins:d}min'] = xr.DataArray(stdWSPD_10N, dims = ['count'], 
                                                attrs = {'units' : 'm/s', 'long_name': f'{timeWindowInMins} min centered std. dev.'})
    
    dummyDS[f'mean Ux_10N {timeWindowInMins:d}min'] = xr.DataArray(meanUx_10N, dims = ['count'], 
                                                attrs = {'units' : 'm/s', 'long_name': f'{timeWindowInMins} min centered mean'})
    dummyDS[f'std. dev. Ux_10N {timeWindowInMins:d}min'] = xr.DataArray(stdUx_10N, dims = ['count'], 
                                                attrs = {'units' : 'm/s', 'long_name': f'{timeWindowInMins} min centered std. dev.'})
    
    dummyDS[f'mean Vy_10N {timeWindowInMins:d}min'] = xr.DataArray(meanVy_10N, dims = ['count'], 
                                                attrs = {'units' : 'm/s', 'long_name': f'{timeWindowInMins} min centered mean'})
    dummyDS[f'std. dev. Vy_10N {timeWindowInMins:d}min'] = xr.DataArray(stdVy_10N, dims = ['count'], 
                                                attrs = {'units' : 'm/s', 'long_name': f'{timeWindowInMins} min centered std. dev.'})
    
    dummyDS[f'mean WDIR {timeWindowInMins:d}min'] = xr.DataArray(meanWDIR, dims = ['count'], 
                                                attrs = {'units' : 'degrees (True)', 'long_name': f'{timeWindowInMins} min centered mean'})
    dummyDS[f'std. dev. WDIR {timeWindowInMins:d}min'] = xr.DataArray(stdWDIR, dims = ['count'], 
                                                attrs = {'units' : 'degrees (True)', 'long_name': f'{timeWindowInMins} min centered std. dev'})
    
    dummyDS[f'mean cosWDIR {timeWindowInMins:d}min'] = xr.DataArray(meanCosWDIR, dims = ['count'], 
                                                attrs = {'units' : '', 'long_name': f'{timeWindowInMins} min centered mean'})
    dummyDS[f'std. dev. cosWDIR {timeWindowInMins:d}min'] = xr.DataArray(stdCosWDIR, dims = ['count'], 
                                                attrs = {'units' : '', 'long_name': f'{timeWindowInMins} min centered std. dev'})
    
    dummyDS[f'mean sinWDIR {timeWindowInMins:d}min'] = xr.DataArray(meanSinWDIR, dims = ['count'], 
                                                attrs = {'units' : '', 'long_name': f'{timeWindowInMins} min centered mean'})
    dummyDS[f'std. dev. sinWDIR {timeWindowInMins:d}min'] = xr.DataArray(stdSinWDIR, dims = ['count'], 
                                                attrs = {'units' : '', 'long_name': f'{timeWindowInMins} min centered std. dev'})
    return dummyDS

def setMeanAndStdXarrVars(ds, indx, timeWindowInMins):
    dummyDS = ds.isel(count = indx)
    
    meanWSPD, stdWSPD, meanWSPD_10N, stdWSPD_10N, stdUx_10N, meanUx_10N, stdVy_10N, meanVy_10N, meanWDIR, stdWDIR, meanCosWDIR, stdCosWDIR, meanSinWDIR, stdSinWDIR = getMeanAndStdDev(dummyDS, timeWindowInMins)

    ds[f'mean WSPD {timeWindowInMins:d}min'][indx] = meanWSPD
    ds[f'std. dev. WSPD {timeWindowInMins:d}min'][indx] = stdWSPD 
                                                
    ds[f'mean WSPD_10N {timeWindowInMins:d}min'][indx] = meanWSPD_10N 
    ds[f'std. dev. WSPD_10N {timeWindowInMins:d}min'][indx] = stdWSPD_10N 
                                                
    
    ds[f'mean Ux_10N {timeWindowInMins:d}min'][indx] = meanUx_10N 
    ds[f'std. dev. Ux_10N {timeWindowInMins:d}min'][indx] = stdUx_10N 

    
    ds[f'mean Vy_10N {timeWindowInMins:d}min'][indx] = meanVy_10N 
    ds[f'std. dev. Vy_10N {timeWindowInMins:d}min'][indx] = stdVy_10N 

    
    ds[f'mean WDIR {timeWindowInMins:d}min'][indx] = meanWDIR 
    ds[f'std. dev. WDIR {timeWindowInMins:d}min'][indx] = stdWDIR 

    
    ds[f'mean cosWDIR {timeWindowInMins:d}min'][indx] = meanCosWDIR 
    ds[f'std. dev. cosWDIR {timeWindowInMins:d}min'][indx] = stdCosWDIR 

    
    ds[f'mean sinWDIR {timeWindowInMins:d}min'][indx] = meanSinWDIR                                                 
    ds[f'std. dev. sinWDIR {timeWindowInMins:d}min'][indx] = stdSinWDIR 

    return ds

In [78]:
test = goodDataSet.copy()

In [79]:
meanWSPD = np.zeros( (len(test['count'])), dtype=np.float32) 

In [80]:
meanWSPD.shape

(156953,)

In [81]:
for timeWindowInMins in range(20, 730, 10):
    test = makeEmptyMeanAndStdXarrVars(test, timeWindowInMins)

In [82]:
test

<xarray.Dataset>
Dimensions:                           (count: 156953, TAO_TIME_INDEX: 73)
Coordinates:
  * count                             (count) int64 0 1 2 ... 244385 244386
  * TAO_TIME_INDEX                    (TAO_TIME_INDEX) int64 -36 -35 ... 35 36
Data variables: (12/1039)
    WDIR                              (count, TAO_TIME_INDEX) float32 dask.array<chunksize=(100, 73), meta=np.ndarray>
    WDIR_QC                           (count, TAO_TIME_INDEX) float32 dask.array<chunksize=(100, 73), meta=np.ndarray>
    WDIR_DM                           (count, TAO_TIME_INDEX) |S1 dask.array<chunksize=(100, 73), meta=np.ndarray>
    WSPD                              (count, TAO_TIME_INDEX) float32 dask.array<chunksize=(100, 73), meta=np.ndarray>
    WSPD_QC                           (count, TAO_TIME_INDEX) float32 dask.array<chunksize=(100, 73), meta=np.ndarray>
    WSPD_DM                           (count, TAO_TIME_INDEX) |S1 dask.array<chunksize=(100, 73), meta=np.ndarray>
    ...                                ...
    mean WDIR 720min                  (count) float32 0.0 0.0 0.0 ... 0.0 0.0
    std. dev. WDIR 720min             (count) float32 0.0 0.0 0.0 ... 0.0 0.0
    mean cosWDIR 720min               (count) float32 0.0 0.0 0.0 ... 0.0 0.0
    std. dev. cosWDIR 720min          (count) float32 0.0 0.0 0.0 ... 0.0 0.0
    mean sinWDIR 720min               (count) float32 0.0 0.0 0.0 ... 0.0 0.0
    std. dev. sinWDIR 720min          (count) float32 0.0 0.0 0.0 ... 0.0 0.0

In [83]:
timeWindowInMins = 720
test[f'mean WSPD {timeWindowInMins:d}min'][1000] = 5

In [86]:
def getStats(ds, indx):
    for timeWindowInMins in range(20, 50, 10):
        ds = setMeanAndStdXarrVars(ds, indx, timeWindowInMins)
    return ds

In [66]:
import multiprocessing
from functools import partial

In [67]:
nprocs = multiprocessing.cpu_count()
print(f'The number of processors is {nprocs}')

The number of processors is 36


In [94]:
partial_do = partial(getStats, ds = test)
p = multiprocessing.Pool(nprocs)

Process ForkPoolWorker-78:
Process ForkPoolWorker-93:
Process ForkPoolWorker-99:
Process ForkPoolWorker-77:
Process ForkPoolWorker-83:
Process ForkPoolWorker-79:
Process ForkPoolWorker-84:
Process ForkPoolWorker-75:
Process ForkPoolWorker-104:
Process ForkPoolWorker-108:
Process ForkPoolWorker-96:
Process ForkPoolWorker-105:
Process ForkPoolWorker-98:
Process ForkPoolWorker-90:
Process ForkPoolWorker-86:
Process ForkPoolWorker-73:
Process ForkPoolWorker-89:
Process ForkPoolWorker-94:
Process ForkPoolWorker-102:
Process ForkPoolWorker-106:
Process ForkPoolWorker-103:
Process ForkPoolWorker-95:
Process ForkPoolWorker-107:
Process ForkPoolWorker-82:
Process ForkPoolWorker-87:
Process ForkPoolWorker-91:
Process ForkPoolWorker-101:
Process ForkPoolWorker-74:
Process ForkPoolWorker-92:
Process ForkPoolWorker-81:
Process ForkPoolWorker-88:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most

In [93]:
test

<xarray.Dataset>
Dimensions:                           (count: 156953, TAO_TIME_INDEX: 73)
Coordinates:
  * count                             (count) int64 0 1 2 ... 244385 244386
  * TAO_TIME_INDEX                    (TAO_TIME_INDEX) int64 -36 -35 ... 35 36
Data variables: (12/1039)
    WDIR                              (count, TAO_TIME_INDEX) float32 dask.array<chunksize=(100, 73), meta=np.ndarray>
    WDIR_QC                           (count, TAO_TIME_INDEX) float32 dask.array<chunksize=(100, 73), meta=np.ndarray>
    WDIR_DM                           (count, TAO_TIME_INDEX) |S1 dask.array<chunksize=(100, 73), meta=np.ndarray>
    WSPD                              (count, TAO_TIME_INDEX) float32 dask.array<chunksize=(100, 73), meta=np.ndarray>
    WSPD_QC                           (count, TAO_TIME_INDEX) float32 dask.array<chunksize=(100, 73), meta=np.ndarray>
    WSPD_DM                           (count, TAO_TIME_INDEX) |S1 dask.array<chunksize=(100, 73), meta=np.ndarray>
    ...                                ...
    mean WDIR 720min                  (count) float32 0.0 0.0 0.0 ... 0.0 0.0
    std. dev. WDIR 720min             (count) float32 0.0 0.0 0.0 ... 0.0 0.0
    mean cosWDIR 720min               (count) float32 0.0 0.0 0.0 ... 0.0 0.0
    std. dev. cosWDIR 720min          (count) float32 0.0 0.0 0.0 ... 0.0 0.0
    mean sinWDIR 720min               (count) float32 0.0 0.0 0.0 ... 0.0 0.0
    std. dev. sinWDIR 720min          (count) float32 0.0 0.0 0.0 ... 0.0 0.0

In [96]:
indexList = list(np.arange(72))

In [97]:
p.map(partial_do, indexList)

KeyboardInterrupt: 

In [ ]:
test

In [60]:
withStats = goodDataSet.map_blocks(getStats, template=goodDataSet).compute()

KeyboardInterrupt: 

In [46]:
WSPD

<xarray.DataArray 'WSPD' (count: 156953, TAO_TIME_INDEX: 73)>
dask.array<xarray-WSPD, shape=(156953, 73), dtype=float32, chunksize=(100, 73), chunktype=numpy.ndarray>
Coordinates:
  * count           (count) int64 0 1 2 3 4 ... 244383 244384 244385 244386
  * TAO_TIME_INDEX  (TAO_TIME_INDEX) int64 -36 -35 -34 -33 -32 ... 33 34 35 36
Attributes:
    standard_name:         wind_speed
    long_name:             Wind Speed
    units:                 meters/second
    valid_min:             0.0
    valid_max:             20.0
    accuracy:              0.3 meters/second or 3%
    resolution:            0.2
    sensor_name:           Wind Propeller [R.M. Young/05103]
    sensor_serial_number:  WIND-11280
    ancillary_variables:   WSPD_QC WSPD_DM
    cell_methods:          TIME: mean (interval: 2 minutes) HEIGHT:point LATI...
    coordinates:           TIME HEIGHT LATITUDE LONGITUDE

In [ ]:
# ndata = len(goodDataSet['count'])

# stats = []
# for indx in range(ndata):
#     if indx%100 == 0:
#         print(indx/ndata * 100)
#     dummyDS = goodDataSet.isel(count = [indx])
#     for timeWindowInMins in range(60, 730, 60):
#         dummyDS = makeMeanAndStdXarrVars(dummyDS, timeWindowInMins)
#         #break
#     stats.append(dummyDS)

0.0
0.06371334093645868
0.12742668187291736
0.19114002280937606
0.2548533637458347
0.3185667046822934
0.3822800456187521
0.44599338655521076
0.5097067274916695
0.5734200684281282
0.6371334093645868
0.7008467503010455
0.7645600912375042
0.828273432173963
0.8919867731104215
0.9557001140468803
1.019413454983339
1.0831267959197977
1.1468401368562564
1.2105534777927152
1.2742668187291737
1.3379801596656324
1.401693500602091
1.4654068415385497
1.5291201824750085
1.592833523411467
1.656546864347926
1.7202602052843845
1.783973546220843
1.847686887157302
1.9114002280937605
1.9751135690302193
2.038826909966678
2.102540250903137
2.1662535918395953
2.229966932776054
2.293680273712513
2.3573936146489713
2.4211069555854303
2.484820296521889
2.5485336374583474
2.6122469783948064
2.675960319331265
2.7396736602677234
2.803387001204182
2.8671003421406405
2.9308136830770994
2.994527024013558
3.058240364950017
3.1219537058864755
3.185667046822934
3.249380387759393
3.313093728695852
3.37680706963231
3.4405

In [ ]:
statsDS = xr.concat(stats, dims='count')

In [ ]:
statsDS.to_netCDF('rainFlagRemovedStatsDatasetAllTAOlocs.nc')

In [ ]:
goodDataSet = goodDataSet.rename_vars({'WSPD':'Wind Speed (TAO)',
                  'WDIR':'Wind Direction (TAO)',
                  'UWND': 'Zonal Wind Speed (TAO)',
                  'VWND': 'Meridional Wind Speed (TAO)',
                  'SST': 'Sea Surface Temperature (TAO)',
                  'RELH': 'Relative Humidity (TAO)',
                  'AIRT': 'Air Temperature (TAO)',
                  'WSPD_10N': 'Neutral Wind Speed at 10m (TAO)',
                  'WSPD_10': 'Wind Speed at 10m (TAO)',
                  'retrieved_wind_speed': 'Wind Speed (QuikSCAT)',
                  'retrieved_wind_direction': 'Wind Direction (QuikSCAT)'})

In [ ]:
goodDataSet.to_netcdf('goodData_QSwithTAO.nc')

In [ ]:
centerData = allDS.sel(TAO_TIME_INDEX = 0)
centerData = centerData.drop('TAO_TIME_INDEX')
indices = list(np.nonzero(~rainMask)[0])

goodData = centerData.isel(count=~rainMask)
goodData = goodData.isel(count = goodData["WSPD_QC"].isin([1,2]))
goodData = goodData.isel(count = goodData["WDIR_QC"].isin([1,2]))
goodData = goodData.isel(count = goodData["SST_QC"].isin([1,2]))
goodData = goodData.isel(count = goodData["RELH_QC"].isin([1,2]))
goodData = goodData.isel(count = goodData["AIRT_QC"].isin([1,2]))


print(len(goodData['count']))




In [ ]:
mask = np.logical_or(goodData['WSPD'] > 1000, goodData['WDIR'] > 3600)
#mask = np.logical_or(mask, abs(goodData['WSPD_10N'].to_numpy()) < 0.1)
mask = np.logical_or(mask, np.isnan(goodData['WSPD'].to_numpy()))
mask = np.logical_or(mask, np.isnan(goodData['WDIR'].to_numpy()))
mask = np.logical_or(mask, np.isnan(goodData['retrieved_wind_speed'].to_numpy()))
mask = np.logical_or(mask, np.isnan(goodData['retrieved_wind_direction'].to_numpy()))
print(np.sum(mask))
df = goodData.isel(count = ~mask)
#df = df.to_dataframe()

In [ ]:
df

In [ ]:
df = df.rename_vars({'WSPD':'Wind Speed (TAO)',
                  'WDIR':'Wind Direction (TAO)',
                  'UWND': 'Zonal Wind Speed (TAO)',
                  'VWND': 'Meridional Wind Speed (TAO)',
                  'SST': 'Sea Surface Temperature (TAO)',
                  'RELH': 'Relative Humidity (TAO)',
                  'AIRT': 'Air Temperature (TAO)',
                  'WSPD_10N': 'Neutral Wind Speed at 10m (TAO)',
                  'WSPD_10': 'Wind Speed at 10m (TAO)',
                  'retrieved_wind_speed': 'Wind Speed (QuikSCAT)',
                  'retrieved_wind_direction': 'Wind Direction (QuikSCAT)'})

In [ ]:
df['Speed Difference (QuikSCAT - TAO)'] = df['Wind Speed (QuikSCAT)'] - df['Neutral Wind Speed at 10m (TAO)']

## change true degrees to degrees where 0 deg mean east and 90 deg mean north 
df['Wind Direction (TAO)'] = (-(df['Wind Direction (TAO)'] - 90.0) + 360)%360
df['Wind Direction (QuikSCAT)'] = (-(df['Wind Direction (QuikSCAT)'] - 90.0) + 360)%360

## calculate zonal and meridional winds 
df['Zonal Neutral Wind Speed (TAO)'] = df['Neutral Wind Speed at 10m (TAO)'] * np.cos(np.deg2rad(df['Wind Direction (TAO)']))
df['Meridional Neutral Wind Speed (TAO)'] = df['Neutral Wind Speed at 10m (TAO)'] * np.sin(np.deg2rad(df['Wind Direction (TAO)']))

df['Zonal Wind Speed (QuikSCAT)'] = df['Wind Speed (QuikSCAT)'] * np.cos(np.deg2rad(df['Wind Direction (QuikSCAT)']))
df['Meridional Wind Speed (QuikSCAT)'] = df['Wind Speed (QuikSCAT)'] * np.sin(np.deg2rad(df['Wind Direction (QuikSCAT)']))

## direction diff in range (-180,180)
dirDiff = ((df['Wind Direction (QuikSCAT)'] - df['Wind Direction (TAO)'])+360)%360
dirDiff = xr.where(dirDiff > 180, dirDiff-360, dirDiff)
df['Direction Difference (QuikSCAT - TAO)'] = dirDiff

df['Zonal Wind Speed Difference (QuikSCAT - TAO)'] = df['Zonal Wind Speed (QuikSCAT)'] - df['Zonal Neutral Wind Speed (TAO)']
df['Meridional Wind Speed Difference (QuikSCAT - TAO)'] = df['Meridional Wind Speed (QuikSCAT)'] - df['Meridional Neutral Wind Speed (TAO)']

for timeWindow in range(20,250,10):
    df[f'Speed Difference (QuikSCAT - TAO {timeWindow} min mean)'] = df['Wind Speed (QuikSCAT)'] - df[f'mean WSPD_10N {timeWindow}min']

    ## change true degrees to degrees where 0 deg mean east and 90 deg mean north 
    df[f'mean WDIR {timeWindow}min'] = (-(df[f'mean WDIR {timeWindow}min'] - 90.0) + 360)%360

    ## direction diff in range (-180,180)
    dirDiff = ((df['Wind Direction (QuikSCAT)'] - df[f'mean WDIR {timeWindow}min'])+360)%360
    dirDiff = xr.where(dirDiff > 180, dirDiff-360, dirDiff)
    df[f'Direction Difference (QuikSCAT - TAO {timeWindow} min mean)'] = dirDiff

    ### 
    df[f'Zonal Wind Speed Difference (QuikSCAT - TAO {timeWindow} min mean)'] = df['Zonal Wind Speed (QuikSCAT)'] - df[f'mean Ux_10N {timeWindow}min']
    df[f'Meridional Wind Speed Difference (QuikSCAT - TAO {timeWindow} min mean)'] = df['Zonal Wind Speed (QuikSCAT)'] - df[f'mean Vy_10N {timeWindow}min']

In [ ]:
df

In [ ]:
df.to_netcdf('rainFlagRemovedBuoyDataBadQualityRemovedMatchup.nc')

In [ ]:
list(df.variables.keys())

In [ ]:
len(df['count'])

In [ ]:
selAllDS = allDS.sel(count=df['count'])

In [ ]:
selAllDS = selAllDS.rename_vars({'WSPD':'Wind Speed (TAO)',
                  'WDIR':'Wind Direction (TAO)',
                  'UWND': 'Zonal Wind Speed (TAO)',
                  'VWND': 'Meridional Wind Speed (TAO)',
                  'SST': 'Sea Surface Temperature (TAO)',
                  'RELH': 'Relative Humidity (TAO)',
                  'AIRT': 'Air Temperature (TAO)',
                  'WSPD_10N': 'Neutral Wind Speed at 10m (TAO)',
                  'WSPD_10': 'Wind Speed at 10m (TAO)',
                  'retrieved_wind_speed': 'Wind Speed (QuikSCAT)',
                  'retrieved_wind_direction': 'Wind Direction (QuikSCAT)'})

In [ ]:
selAllDS['Speed Difference (QuikSCAT - TAO)'] = selAllDS['Wind Speed (QuikSCAT)'] - selAllDS['Wind Speed (TAO)']

selAllDS['Wind Direction (TAO)'] = (-(selAllDS['Wind Direction (TAO)'] - 90.0) + 360)%360
selAllDS['Wind Direction (QuikSCAT)'] = (-(selAllDS['Wind Direction (QuikSCAT)'] - 90.0) + 360)%360

selAllDS['Direction Difference (QuikSCAT - TAO 30 min mean)'] = selAllDS['Wind Direction (QuikSCAT)'] - selAllDS['mean WSPD 30min']
selAllDS['Direction Difference (QuikSCAT - TAO 30 min mean)'] = selAllDS['Wind Direction (QuikSCAT)'] - selAllDS['mean WDIR 30min']

selAllDS['Direction Difference (QuikSCAT - TAO 1 hr mean)'] = selAllDS['Wind Direction (QuikSCAT)'] - selAllDS['mean WSPD 60min']
selAllDS['Direction Difference (QuikSCAT - TAO 1 hr mean)'] = selAllDS['Wind Direction (QuikSCAT)'] - selAllDS['mean WDIR 60min']

selAllDS['Direction Difference (QuikSCAT - TAO 2 hr mean)'] = selAllDS['Wind Direction (QuikSCAT)'] - selAllDS['mean WSPD 120min']
selAllDS['Direction Difference (QuikSCAT - TAO 2 hr mean)'] = selAllDS['Wind Direction (QuikSCAT)'] - selAllDS['mean WDIR 120min']


selAllDS['Zonal Neutral Wind Speed at 10m (TAO)'] = selAllDS['Neutral Wind Speed at 10m (TAO)']*np.cos(np.deg2rad(selAllDS['Wind Direction (TAO)']))
selAllDS['Meridional Neutral Wind Speed at 10m (TAO)'] = selAllDS['Neutral Wind Speed at 10m (TAO)']*np.sin(np.deg2rad(selAllDS['Wind Direction (TAO)']))

selAllDS['Zonal Neutral Wind Speed at 10m (QuikSCAT)'] = selAllDS['Wind Speed (QuikSCAT)']*np.cos(np.deg2rad(selAllDS['Wind Direction (QuikSCAT)']))
selAllDS['Meridional Neutral Wind Speed at 10m (QuikSCAT)'] = selAllDS['Wind Speed (QuikSCAT)']*np.sin(np.deg2rad(selAllDS['Wind Direction (QuikSCAT)']))

selAllDS['Zonal Wind Speed Difference (QuikSCAT - TAO)'] = selAllDS['Zonal Neutral Wind Speed at 10m (QuikSCAT)'] - selAllDS['Zonal Neutral Wind Speed at 10m (TAO)']
selAllDS['Meridional Wind Speed Difference (QuikSCAT - TAO)'] = selAllDS['Meridional Neutral Wind Speed at 10m (QuikSCAT)'] - selAllDS['Meridional Neutral Wind Speed at 10m (TAO)']


In [ ]:
selAllDS